# TiDB Python SDK

A powerful Python SDK for vector storage and retrieval operations with TiDB.

- 🔄 Automatic embedding generation
- 🔍 Vector similarity search
- 🎯 Advanced filtering capabilities
- 📦 Bulk operations support

## Installation

In [1]:
%pip install -q pytidb "pytidb[models]" pandas

Note: you may need to restart the kernel to use updated packages.


## Configuration

- Go [tidbcloud.com](https://tidbcloud.com/) or using [tiup playground](https://docs.pingcap.com/tidb/stable/tiup-playground/) to create a free TiDB database cluster.
- Go [OpenAI platform](https://platform.openai.com/api-keys) to create your API key.

Configuration can be provided through environment variables, or using `.env`:

In [2]:
%%bash

# Check if the .env file is existing.
if [ -f .env ]; then
    exit 0
fi

# Create .env file with your configuration.
cat > .env <<EOF
TIDB_HOST=localhost
TIDB_PORT=4000
TIDB_USERNAME=root
TIDB_PASSWORD=
TIDB_DATABASE=test
OPENAI_API_KEY='your_openai_api_key'
EOF

In [3]:
import dotenv

dotenv.load_dotenv()

True

## Quickstart

### Connect to TiDB

In [4]:
import os
from pytidb import TiDBClient

db = TiDBClient.connect(
    host=os.getenv("TIDB_HOST"),
    port=int(os.getenv("TIDB_PORT")),
    username=os.getenv("TIDB_USERNAME"),
    password=os.getenv("TIDB_PASSWORD"),
    database=os.getenv("TIDB_DATABASE"),
)

# If you are using DATABASE_URL
# db = TiDBClient.connect(database_url=os.getenv("DATABASE_URL"))

### Create table

In [5]:
from pytidb.schema import TableModel, Field
from pytidb.embeddings import EmbeddingFunction

# Define your embedding model.
text_embed = EmbeddingFunction("openai/text-embedding-3-small")


class Chunk(TableModel, table=True):
    __tablename__ = "chunks"
    __table_args__ = {"extend_existing": True}

    id: int = Field(primary_key=True)
    text: str = Field()
    text_vec: list[float] = text_embed.VectorField(
        source_field="text"
    )  # 👈 Define the vector field.
    user_id: int = Field()


table = db.create_table(schema=Chunk)

### Insert Data

🔢 Auto embedding: when you insert new data, the SDK automatically embeds the corpus for you.

In [6]:
table.truncate()
table.insert(
    Chunk(text="The quick brown fox jumps over the lazy dog", user_id=1),
)
table.bulk_insert(
    [
        Chunk(text="A quick brown dog runs in the park", user_id=2),
        Chunk(text="The lazy fox sleeps under the tree", user_id=2),
        Chunk(text="A dog and a fox play in the park", user_id=3),
    ]
)
table.rows()

4

### Vector Search

In [7]:
res = (
    table.search(
        "A quick fox in the park"
    )  # 👈 The query will be embedding automatically.
    .filter({"user_id": 2})
    .limit(2)
    .to_pandas()
)
res

,id,text,text_vec,user_id,_distance,similarity_score,score
0,2,A quick brown dog runs in the park,"[-0.04131943, -0.009358937, 0.012413035, -0.03...",2,0.334548,0.665452,0.665452
1,3,The lazy fox sleeps under the tree,"[-0.016022757, -0.0026831497, -0.017942838, -0...",2,0.445237,0.554763,0.554763


### Advanced Filtering

TiDB Client supports various filter operators for flexible querying:

| Operator | Description | Example |
|----------|-------------|---------|
| `$eq` | Equal to | `{"field": {"$eq": "hello"}}` |
| `$gt` | Greater than | `{"field": {"$gt": 1}}` |
| `$gte` | Greater than or equal | `{"field": {"$gte": 1}}` |
| `$lt` | Less than | `{"field": {"$lt": 1}}` |
| `$lte` | Less than or equal | `{"field": {"$lte": 1}}` |
| `$in` | In array | `{"field": {"$in": [1, 2, 3]}}` |
| `$nin` | Not in array | `{"field": {"$nin": [1, 2, 3]}}` |
| `$and` | Logical AND | `{"$and": [{"field1": 1}, {"field2": 2}]}` |
| `$or` | Logical OR | `{"$or": [{"field1": 1}, {"field2": 2}]}` |


In [8]:
chunks = table.query({"user_id": 1})
[(c.id, c.text, c.user_id) for c in chunks]

[(1, 'The quick brown fox jumps over the lazy dog', 1)]

### Multiple Tables Join

In [9]:
# Create a table to store user data:
class User(TableModel, table=True):
    __tablename__ = "users"
    __table_args__ = {"extend_existing": True}

    id: int = Field(primary_key=True)
    name: str = Field(max_length=20)


user_table = db.create_table(schema=User)

In [10]:
user_table.truncate()
user_table.insert(User(id=1, name="Alice"))

User(name='Alice', id=1)

In [11]:
from pytidb import Session
from pytidb.sql import select

db_engine = db.db_engine
with Session(db_engine) as db_session:
    query = (
        select(Chunk).join(User, Chunk.user_id == User.id).where(User.name == "Alice")
    )
    chunks = db_session.exec(query).all()

[(c.id, c.text, c.user_id) for c in chunks]

[(1, 'The quick brown fox jumps over the lazy dog', 1)]

### Update Data

In [12]:
old_chunk = table.query({"text": "A quick brown dog runs in the park"})[0]
chunk_id = old_chunk.id
(old_chunk.text, old_chunk.text_vec)

('A quick brown dog runs in the park',
 array([-0.04131943, -0.00935894,  0.01241304, ..., -0.00588697,
        -0.00739252,  0.01385098], dtype=float32))

In [13]:
table.update(
    values={
        "text": "foo"  # 👈 Only provide the fields you want to update.
    },
    filters={"id": chunk_id},
)

In [14]:
new_chunk = table.get(chunk_id)
(new_chunk.text, new_chunk.text_vec)  # 👈 The vector field is updated automatically.

('foo',
 array([-0.00584331, -0.0131113 , -0.01458535, ..., -0.00907705,
        -0.0073632 , -0.01883118], dtype=float32))

### Delete Data

In [15]:
table.rows()

4

In [16]:
table.delete(filters={"user_id": 2})

In [17]:
table.rows()

2

### Execute raw SQL

- Using `execute()` to execute INSERT / UPDATE / DELETE statement.
- Using `query()` to execute SELECT / SHOW statement.

#### Execute SQL

The `execute()` method will return a `SQLExecuteResult` instance.

In [18]:
db.execute("INSERT INTO chunks(text, user_id) VALUES ('inserted from raw sql', 5)")

SQLExecuteResult(rowcount=1, success=True, message=None)

Both of the `execute` and `query` methods are support the **Parameterized SQL** feature, which help you avoid [SQL injection](https://en.wikipedia.org/wiki/SQL_injection) while using the dynamic SQL feature.

In [19]:
db.execute(
    "INSERT INTO chunks(text, user_id) VALUES (:text, :user_id)",
    {"text": "inserted from dynamic sql", "user_id": 6},
)

SQLExecuteResult(rowcount=1, success=True, message=None)

#### Query SQL

The `query()` method will return a `SQLQueryResult` instance with some helper methods:

- `to_pandas()`
- `to_list()`
- `to_rows()`
- `scalar()`

The `to_pandas()` method to convert the query result to a `pandas.DataFrame`, which is displayed as human-friendly style on the notebook:

In [20]:
db.query("SELECT id, text, user_id FROM chunks").to_pandas()

,id,text,user_id
0,1,The quick brown fox jumps over the lazy dog,1
1,4,A dog and a fox play in the park,3
2,5,inserted from raw sql,5
3,6,inserted from dynamic sql,6


The `to_list()` method will convert the query result into a list of dict.

In [21]:
db.query(
    "SELECT id, text, user_id FROM chunks WHERE user_id = :user_id", {"user_id": 3}
).to_list()

[{'id': 4, 'text': 'A dog and a fox play in the park', 'user_id': 3}]

The `to_rows` method will return a list of tuple, every tuple represent of one row of data.

In [22]:
db.query("SHOW TABLES;").to_rows()

[('chunks',), ('users',)]

In [23]:
db.query("SELECT COUNT(*) FROM chunks;").scalar()

4

### Table Management

#### List tables

In [24]:
db.table_names()

['chunks', 'users']

#### Truncate table

Clear all data in the table:

In [25]:
table.truncate()
table.rows()

0

#### Drop table


In [26]:
db.drop_table("chunks")

SQLExecuteResult(rowcount=0, success=True, message=None)